In [ ]:
import pickle
import polars as pl
import numpy as np

In [ ]:
with open('./data.pkl', 'rb') as f:
    df = pickle.load(f)

df

In [ ]:
df_mod = df.with_columns(trajectory_id=pl.Series(values=[*range(len(df))]))#.filter(pl.col('trajectory_id') == 763)

In [ ]:
import time

# Old

In [ ]:
st = time.time()
df_flatten = (df_mod.select(
        pl.col('trajectory_id'),
        pl.col('timestamp'),
        pl.col('downstream_av_id'),
        pl.col('distance_to_downstream_av_meters'),
    )
    .filter(pl.col('downstream_av_id').list.len() != 0)
    .with_columns(
        timestamp_int=pl.col('timestamp')
            .map_elements(lambda ts: [int(ts[0] * 100 + 0.5) + int((t - ts[0]) * 100 + 0.5) for t in ts],
                                                       return_dtype=list[int]),
        rank=pl.col('timestamp')
            .map_elements(lambda ts: list(range(len(ts))),
                                              return_dtype=list[int]),
    )
    # int
    .explode(
        'timestamp',
        'timestamp_int',
        'downstream_av_id',
        'distance_to_downstream_av_meters',
        'rank',
    )
    .drop_nulls('downstream_av_id')
)
df_flatten.sort('rank').filter((1040 < pl.col.rank) & (pl.col.rank < 1050))

In [ ]:
import itertools
def interpolate(arr):
    last = arr[-1]
    arr = np.array(arr)
    prv = arr[:-1]
    nxt = arr[1:]
    dff = nxt - prv
    dff25 = prv + dff * .25
    dff50 = prv + dff * .50
    dff75 = prv + dff * .75
    return list(np.stack((prv, dff25, dff50, dff75), axis=-1).flatten()) + [last]

def fillnone(arr, nones):
    chunks = []
    for idx, prv, nxt in reversed(nones):
        dist = nxt - prv
        assert dist % 4 == 0, (nxt, prv)
        chunk = arr[idx+1:]
        arr = arr[:idx+1]
        chunks.append(chunk)
    # chunks.append(arr)
    chunks.reverse()

    return list(itertools.chain(*itertools.chain([arr], *((([float('nan')] * ((nxt - prv) // 4 - 1)), chunk) for (_, prv, nxt), chunk in zip(nones, chunks)))))
def fillnone2(arr, nones):
    chunks = []
    for idx, prv, nxt in reversed(nones):
        dist = nxt - prv
        assert dist % 4 == 0, (nxt, prv)
        chunk = arr[idx+1:]
        arr = arr[:idx+1]
        chunks.append(chunk)
    # chunks.append(arr)
    chunks.reverse()

    return list(itertools.chain(*itertools.chain([arr], *((([None] * ((nxt - prv) // 4 - 1)), chunk) for (_, prv, nxt), chunk in zip(nones, chunks)))))
    

df_flatten_interpolate = (df_flatten
    .group_by('trajectory_id', 'downstream_av_id')
    .all()
    .with_columns(
        nones=pl.col('timestamp_int').map_elements(lambda ts: [(idx, prv, nxt) for idx, (prv, nxt) in enumerate(zip(ts[:-1], ts[1:])) if nxt - prv != 4], return_dtype=list[tuple[int, int, int]])
    )
    # .filter(c.nones.list.len() != 0)
    .with_columns(
        timestamp=pl.struct('nones', 'timestamp').map_elements(lambda r: fillnone(r['timestamp'], r['nones']), return_dtype=list[float]),
        distance_to_downstream_av_meters=pl.struct('nones', 'distance_to_downstream_av_meters').map_elements(lambda r: fillnone(r['distance_to_downstream_av_meters'], r['nones']), return_dtype=list[float]),
        timestamp_int=pl.struct('nones', 'timestamp_int').map_elements(lambda r: fillnone2(r['timestamp_int'], r['nones']), return_dtype=list[int]),
        rank=pl.struct('nones', 'rank').map_elements(lambda r: fillnone2(r['rank'], r['nones']), return_dtype=list[int]),
    )
    .with_columns(
        timestamp=pl.col('timestamp').map_elements(interpolate, return_dtype=list[float]),
        distance_to_downstream_av_meters=pl.col('distance_to_downstream_av_meters').map_elements(interpolate, return_dtype=list[float]),
        timestamp_int=pl.col('timestamp_int').map_elements(lambda ts: list(range(ts[0], ts[-1] + 1)), return_dtype=list[int]),
        rank=pl.col('rank').map_elements(lambda r: range(r[0] * 4, r[-1] * 4 + 1), return_dtype=list[int]),
    )
    # .with_columns(
    #     timestamp_len=pl.col('timestamp').list.len(),
    #     distance_to_downstream_av_meters_len=pl.col('distance_to_downstream_av_meters').list.len(),
    #     # timestamp_int=pl.col('timestamp_int').map_elements(lambda ts: len(list(range(ts[0], ts[-1] + 1))), return_dtype=int),
    #     # rank=pl.col('rank').map_elements(lambda r: len(range(r[0], r[0] + (len(r) - 1) * 4 + 1)), return_dtype=int),
    # )
    .drop('nones')
    .explode(
        'timestamp',
        'distance_to_downstream_av_meters',
        'timestamp_int',
        'rank',
    )
    # .drop_nans('timestamp')
    .filter(
        ~pl.col('timestamp').is_nan()
    )
    # .filter(
    #     (pl.col('timestamp').list.len() != pl.col('rank').list.len()) |
    #     (pl.col('timestamp').list.len() != pl.col('timestamp_int').list.len()) |
    #     (pl.col('timestamp').list.len() != pl.col('distance_to_downstream_av_meters').list.len())
    # )
)#['timestamp_int'][0]
# with pl.Config(tbl_rows=30):
#     print(df_flatten_interpolate.sort('rank', 'downstream_av_id').filter((4155 < pl.col('rank'))&(pl.col('rank')<4170)))

en1 = time.time() - st
df_flatten_interpolate

In [ ]:
df_out_1 = (df_flatten_interpolate
    .group_by('downstream_av_id', 'timestamp_int')
    .all()
    .with_columns(
        min_idx=pl.col('distance_to_downstream_av_meters').list.arg_min()
    )
    .with_columns(
        timestamp=pl.col('timestamp').list.get(pl.col('min_idx')),
        trajectory_id=pl.col('trajectory_id').list.get(pl.col('min_idx')),
        distance_to_downstream_av_meters=pl.col('distance_to_downstream_av_meters').list.get(pl.col('min_idx')),
        rank=pl.col('rank').list.get(pl.col('min_idx')),
    )
    .drop('min_idx', 'timestamp_int')
    .filter(pl.col('rank') % 4 == 0)
    .sort('timestamp')
    .group_by('downstream_av_id')
    .all()
)
df_out_1

# New

In [ ]:
# POS = 'downstream'
POS = 'upstream'

def new_av(aid):
    """
    Given a list of av_ids, return a list of indices of when av_id changes.
    """
    aid = np.array([a if a is not None else -1 for a in aid], dtype=int)
    return list(np.argwhere(aid[:-1] != aid[1:])[:, 0] + 1)


df_split_av_ = (df
    .select(
        pl.col('trajectory_id'),
        pl.col('timestamp'),
        pl.col(f'{POS}_av_id'),
        pl.col(f'distance_to_{POS}_av_meters'),
    )
    .rename({'trajectory_id': 'tid', f'{POS}_av_id': 'avid', f'distance_to_{POS}_av_meters': 'dist'})
    # Remove all the trajectories without any av id.
    .filter(pl.col('avid').list.drop_nulls().list.len() != 0)
    .with_columns(
        # Since the timestamps for each trajectories do not align, we snap the timestamp of all the trajectories to the nearest millisecond.
        timestamp_int=pl.col('timestamp')
            .map_elements(lambda ts: [int(ts[0] * 100 + 0.5) + int((t - ts[0]) * 100 + 0.5) for t in ts],
                          return_dtype=list[int]),
        # A list of indices of a trajectory points. Will be used to reference each point of the trajectory.
        trajectory_point_idx=pl.col('timestamp')
            .map_elements(lambda ts: list(range(len(ts))),
                          return_dtype=list[int]),
    )
    .with_columns(
        # Derive when a trajectory changes its up/down-stream AV.
        new_av=pl.col('avid')
            .map_elements(new_av, return_dtype=list[int])
    )
    .with_columns(
        # Split timestamps based on the AV the trajectory follow / lead.
        timestamp=pl.struct('timestamp', 'new_av')
            .map_elements(lambda x: [list(l) for l in np.split(x['timestamp'], x['new_av'])],
                          return_dtype=list[list[float]]),
        # Split avid based on the AV the trajectory follow / lead.
        # Take the first AV id because all the ids in the splitted array are the same (because we split by AV id)
        avid=pl.struct('avid', 'new_av')
            .map_elements(lambda x: [l[0] for l in np.split(x['avid'], x['new_av'])],
                          return_dtype=list[int]),
        # Split dist based on the AV the trajectory follow / lead.
        dist=pl.struct('dist', 'new_av')
            .map_elements(lambda x: [list(l) for l in np.split(x['dist'], x['new_av'])],
                          return_dtype=list[list[float]]),
        # Split timestamp_int based on the AV the trajectory follow / lead.
        timestamp_int=pl.struct('timestamp_int', 'new_av')
            .map_elements(lambda x: [list(l) for l in np.split(x['timestamp_int'], x['new_av'])],
                          return_dtype=list[list[int]]),
        # Split trajectory_point_idx based on the AV the trajectory follow / lead.
        trajectory_point_idx=pl.struct('trajectory_point_idx', 'new_av')
            .map_elements(lambda x: [list(l) for l in np.split(x['trajectory_point_idx'], x['new_av'])],
                          return_dtype=list[list[int]]),
    )
)
df_split_av_

In [ ]:
df_split_av = (df_split_av_
    .drop('new_av')
    .explode(
        # list[list[timestamp]] per row -> list[timestamp] per row
        'timestamp',
        # list[list[timestamp_int]] per row -> list[timestamp_int] per row
        'timestamp_int',
        # list[avid] per row -> avid per row
        'avid',
        # list[list[dist]] per row -> list[dist] per row
        'dist',
        # list[list[trajectory_point_idx]] per row -> list[trajectory_point_idx] per row
        'trajectory_point_idx',
    )
    .filter(~pl.col('avid').is_null())
)
df_split_av

In [ ]:
import itertools
def interpolate(arr):
    last = arr[-1]
    arr = np.array(arr)
    prv = arr[:-1]
    nxt = arr[1:]
    dff = nxt - prv
    dff25 = prv + dff * .25
    dff50 = prv + dff * .50
    dff75 = prv + dff * .75
    return list(np.stack((prv, dff25, dff50, dff75), axis=-1).flatten()) + [last]


df_interpolate_ = (df_split_av
    .with_columns(
        # Points of a trajectories are captured roughly every 4 millisecond and 2 trajectories might not align, so we interpolate trajectories points to every 1 millisecond.
        timestamp_int=pl.col('timestamp_int').map_elements(lambda ts: list(range(ts[0], ts[-1] + 1)), return_dtype=list[int]),
        # Interpolate timestamp
        timestamp=pl.col('timestamp').map_elements(interpolate, return_dtype=list[float]),
        # Interpolate dist
        dist=pl.col('dist').map_elements(interpolate, return_dtype=list[float]),
        # Interpolate trajectory_point_idx
        trajectory_point_idx_interp=pl.col('trajectory_point_idx').map_elements(lambda r: range(r[0] * 4, r[-1] * 4 + 1), return_dtype=list[int]),
        # avid=pl.col('avid').list.first(),
    )
)

df_interpolate_

In [ ]:
df_interpolate = (df_interpolate_
    # Drop the original trajectory_point_idx it has different lenght than other columns (cannot explode)
    .drop('trajectory_point_idx')
    .explode(
        # Flatten all trajectories points
        'timestamp',
        'dist',
        'timestamp_int',
        'trajectory_point_idx_interp',
    )
)
df_interpolate

In [ ]:
df_interpolate_grouped = (df_interpolate
    # For every AV id and every timestamp (snapped to the nearest millissecond -> every millisecond)
    .group_by('avid', 'timestamp_int')
    # Roll up ungrouped columns into lists
    .all()
    .with_columns(
        # Find the index of the nearest vehicle
        nearest_idx=pl.col('dist').list.arg_min()
            if POS == 'downstream'
            else pl.col('dist').list.arg_max()
    )
)
df_interpolate_grouped

In [ ]:
df_nearest = (df_interpolate_grouped
    .with_columns(
        # Choose the vehicle from the index
        timestamp=pl.col('timestamp').list.get(pl.col('nearest_idx')),
        trajectory_id=pl.col('tid').list.get(pl.col('nearest_idx')),
        dist=pl.col('dist').list.get(pl.col('nearest_idx')),
        trajectory_point_idx_interp=pl.col('trajectory_point_idx_interp').list.get(pl.col('nearest_idx')),
    )
    .drop('nearest_idx', 'tid')
    # Only include the non-interpolated trajectory points
    .filter(pl.col('trajectory_point_idx_interp') % 4 == 0)
    .sort('timestamp')
    # (trajectory_id, trajectory_point_idx) can now be used to reference the trajectory points in the original dataframe.
    .with_columns(trajectory_point_idx=pl.col('trajectory_point_idx_interp') // 4)
)
df_nearest

In [ ]:
(df_nearest
    .group_by('avid')
    # Roll up ungrouped columns into lists
    .all()
)

# Comparison

In [ ]:
a = df_flatten_interpolate.rename({'trajectory_id': 'tid', 'downstream_av_id': 'avid', 'distance_to_downstream_av_meters': 'dist', 'rank': 'loc_idx'}).select('tid', 'timestamp_int', 'avid', 'dist', 'loc_idx')
a.sort('tid', 'timestamp_int')

In [ ]:
b = df_interpolate.select('tid', 'timestamp_int', 'avid', 'dist', 'loc_idx')
b.sort('tid', 'timestamp_int')

In [ ]:
b.join(a, on=['tid', 'timestamp_int', 'loc_idx'], how='anti').sort('tid', 'loc_idx', 'timestamp_int')

In [ ]:
a.join(b, on=['tid', 'timestamp_int', 'avid', 'loc_idx'], how='anti').sort('tid', 'loc_idx', 'timestamp_int')

In [ ]:
df_out_1.rename({'downstream_av_id': 'avid', 'distance_to_downstream_av_meters': 'dist', 'rank': 'loc_idx'})

In [ ]:
df_out_2

In [ ]:
join = df_out_1.rename({'downstream_av_id': 'avid', 'distance_to_downstream_av_meters': 'dist', 'rank': 'loc_idx'}).join(df_out_2, on='avid', how='anti')
join

In [ ]:
join = df_out_2.join(df_out_1.rename({'downstream_av_id': 'avid', 'distance_to_downstream_av_meters': 'dist', 'rank': 'loc_idx'}), on='avid', how='anti')
join

In [ ]:
en1, en2